In [1]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

/tmp/ipykernel_54454/1667009573.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [2]:
import os
def list_files(dir):
    r = []
    subdirs = [x[0] for x in os.walk(dir)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]

        if len(files) > 0:
            for file in files:
                r.append(os.path.join(subdir, file))
    return r

In [ ]:
#r = list_files("/media/data1/ravram/DeepOCT_Abbott")
#display(len(r), " files found")
#df = pd.DataFrame(list(zip(r)), columns=["path"])
#new_2 = df["path"].str.split("/", n=8, expand=True)

#df["StudyInstanceUID"] = new_2[6]
#df["patient_id"] = new_2[5]
#df["dicom_id"] = new_2[7]
#display(df.head(n=15))

In [7]:
r = list_files("../Oread/data_Maher")
df = pd.DataFrame(list(zip(r)), columns=["path"])
df = df.loc[df["path"].str.contains("XA")]
display(df)
df.to_csv('data_Maher.csv')

#new_2 = df["path"].str.split("/", n=8, expand=True)

#df["StudyInstanceUID"] = new_2[6]
#df["patient_id"] = new_2[5]
#df["dicom_id"] = new_2[7]
#display(df.head(n=15))

,path
0,../Oread/data_Maher/XA_00019
1,../Oread/data_Maher/XA_00018
4,../Oread/data_Maher/XA_00015
6,../Oread/data_Maher/XA_00005
7,../Oread/data_Maher/XA_00006
8,../Oread/data_Maher/XA_00009
9,../Oread/data_Maher/XA_00003
11,../Oread/data_Maher/XA_00013
12,../Oread/data_Maher/XA_00008
14,../Oread/data_Maher/XA_00002


In [ ]:
df_extracted = pd.read_csv("data/Abbott_OCT/database-c-find_ABBOTT.csv")
df_extracted = pd.DataFrame({"path": r})
df_extracted["FileType"] = df_extracted.path.apply(lambda x: x.split(".")[-1])
display("Total files", df_extracted.FileType.value_counts())

df_extracted = df_extracted.loc[df_extracted["FileType"] == "dcm"]
new_2 = df_extracted["path"].str.split("/", n=7, expand=True)

df_extracted["mrn"] = new_2[5]
df_extracted["StudyInstanceUID"] = new_2[6]
df_extracted["dicom_id"] = new_2[7].str.rstrip(".dcm")
df_extracted_study_level = (
    df_extracted.groupby(["mrn", "StudyInstanceUID"]).first().reset_index()
)

In [ ]:
df_extracted_m = pd.merge(
    df_extracted_study_level.drop(columns={"dicom_id", "path"}),
    df,
    how="inner",
    on=["StudyInstanceUID"],
)
display(
    df_extracted_m.loc[
        df_extracted_m["dicom_id"]
        == "1.3.12.2.1107.5.4.5.135214.30000022072511311760100000156.dcm"
    ]
)
df_extracted_m.to_csv("data/Abbott_OCT/df_extracted_m_ABBOTT.csv", index=False)

In [ ]:
df_extracted_m = pd.read_csv("data/Abbott_OCT/df_extracted_m_ABBOTT.csv")
display(
    df_extracted_m.loc[
        df_extracted_m["StudyInstanceUID"] == "2.16.124.113611.1.118.1.1.5884039"
    ]
)

### Extract AVI Metadata




In [1]:
from downloadAvi import extract_avi_metadata as avi_meta

avi_meta.extract_avi_and_metadata("data_Maher.csv")


0it [00:00, ?it/s]

< cv2.VideoWriter 0x7f459fb834d0>

'dicom_avi_extracted/data_Maher_XA_00019.avi'

0it [00:00, ?it/s]


DICOMPath doesn't exist
1 ../Oread/data_Maher/XA_00019


error: OpenCV(4.6.0) /io/opencv/modules/videoio/src/cap_ffmpeg.cpp:192: error: (-215:Assertion failed) image.depth() == CV_8U in function 'write'


In [1]:
import pydicom as dicom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

df = pd.read_csv("data_Maher.csv")

dataset = dicom.dcmread(df.path.iloc[0], force=True)

testarray = dataset.pixel_array
testarray = np.stack((testarray,) * 3, axis=-1)


### Plot images for each class for the OBJECT RECON

In [ ]:
This code will create a 1x11 subplot with each subplot showing the middle frame from a random video of the corresponding class. The title of each subplot is the class name.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from random import choice

# Let's assume df is your dataframe with columns 'FileName' and 'Class'

def plot_middle_frames(df):
    classes = df['y_true_cat'].unique()
    fig, axs = plt.subplots(3, 4, figsize=(20, 20))

    for i, cls in enumerate(classes):
        # Select a random sample from each class
        sample = df[df['y_true_cat'] == cls].sample(1).iloc[0]
        video_path = sample['FileName']

        # Load the video
        cap = cv2.VideoCapture(video_path)

        # Get the number of frames
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Select the middle frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, num_frames // 2)

        # Read the middle frame
        ret, frame = cap.read()

        # Plot the frame
        row = i // 4
        col = i % 4
        axs[row, col].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        axs[row, col].set_title(cls)
        axs[row, col].axis('off')  # Hide axes

    # Show the plot
    plt.tight_layout()
    plt.show()


### 